In [86]:
from scipy import stats
import glob
import os
import re
import pandas as pd
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from datetime import timedelta
import sys
import datetime
import tensorflow as tf
import math
from sklearn import cross_validation
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier as RF
import scipy as sp
from sklearn.decomposition import PCA
import sklearn.metrics
import random
from scipy import stats


In [111]:
LUNA_CHUNKS = '/kaggle_2/luna/luna16/data/pre_processed_chunks_segmented/'
DSB_DATA_PATH = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed/'
LABELS = '/kaggle/dev/data-science-bowl-2017-data/stage1_labels.csv'
PATIENT_SCANS = 'scan_segmented_lungs_fill_'
CHUNK_SIZE = 64
NUM_CLASSES = 7
OVERLAP_PERCENTAGE = 0.6

In [103]:
luna_chunks_x = []
luna_chunks_y = []
for folder in (glob.glob(LUNA_CHUNKS + '*_X.npy'))[0:10]:
    m = re.match('([a-f0-9].*)_X.npy', os.path.basename(folder))
    chunks = np.load(LUNA_CHUNKS + m.group(0))
    y_labels = np.load(LUNA_CHUNKS + m.group(1)+'_Y.npy')
    luna_chunks_x.append(chunks)
    luna_chunks_y.append(y_labels)

In [120]:
dsb_chunks_x = []
patient_ids = []
for folder in (glob.glob(DSB_DATA_PATH + PATIENT_SCANS + '*'))[0:20]:
    m = re.match(PATIENT_SCANS +'([a-f0-9].*).npy', os.path.basename(folder))
    scans = np.load(DSB_DATA_PATH + m.group(0))
    patient_uid = m.group(1)
    patient_ids.append(patient_uid)
    
    
    chunk_counter = 1
    step_size = int((CHUNK_SIZE*(1-OVERLAP_PERCENTAGE)))
    num_chunks_0 = int((scans.shape[0])/(step_size)) + 1
    num_chunks_1 = int((scans.shape[1])/(step_size)) + 1
    num_chunks_2 = int((scans.shape[2])/(step_size)) + 1
    chunk_list = []     
    
    start_index_0 = 0
    end_index_0 = 0
    for i in range(0, num_chunks_0):
        end_index_0 = start_index_0 + CHUNK_SIZE
        
        start_index_1 = 0
        end_index_1 = 0
        for j in range(0, num_chunks_1):
            end_index_1 = start_index_1 + CHUNK_SIZE
                       
            start_index_2 = 0
            end_index_2 = 0
            for k in range(0, num_chunks_2):
                end_index_2 = start_index_2 + CHUNK_SIZE

                end_index_0 = scans.shape[0] if  (end_index_0 > scans.shape[0]) else end_index_0
                end_index_1 = scans.shape[1] if  (end_index_1 > scans.shape[1]) else end_index_1
                end_index_2 = scans.shape[2] if  (end_index_2 > scans.shape[2]) else end_index_2
                                
                chunk = np.full((CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), -1000.0)
                
                end_index_0_chunks = end_index_0 - start_index_0
                end_index_1_chunks = end_index_1 - start_index_1
                end_index_2_chunks = end_index_2 - start_index_2
                
                chunk[0:end_index_0_chunks, 0:end_index_1_chunks, 0:end_index_2_chunks] = scans[start_index_0:end_index_0, start_index_1:end_index_1, start_index_2:end_index_2]
                chunk_list.append(chunk)
                
                chunk_counter += 1        
                start_index_2 += step_size
            start_index_1 += step_size
        start_index_0 += step_size
        
    X = np.ndarray([len(chunk_list), CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE], dtype=np.int16)
    Y = np.zeros([len(chunk_list), NUM_CLASSES], dtype=np.int16)
    for m in range(0,len(chunk_list)):
        X[m,:,:] = chunk_list[m]
    
#     np.save(OUTPUT_FOLDER_ORIGINAL + patient_uid + '_X.npy', X)
#     np.save(OUTPUT_FOLDER_ORIGINAL + patient_uid + '_Y.npy', Y)
    
    print('processed patient:', patient_uid  , '_original shape:', scans.shape )
    print('_num_chunks:', len(chunk_list), '_X.shape:', X.shape, '_Y.shape:', Y.shape)
    dsb_chunks_x.append(X)
    
#     # Normalizing and Zero Centering
#     X_nz = normalize(X)
#     X_nz = zero_center(X_nz)
#     np.save(OUTPUT_FOLDER_NZ + patient_uid + '_X.npy', X_nz)
#     np.save(OUTPUT_FOLDER_NZ + patient_uid + '_Y.npy', Y)
    
    # Clearning memory
    del X,Y


processed patient: 0a0c32c9e08cc2ea76a71649de56be6d _original shape: (332, 340, 340)
_num_chunks: 2744 _X.shape: (2744, 64, 64, 64) _Y.shape: (2744, 7)
processed patient: 7577cbd6961b0cab27f88727dcd2d6d3 _original shape: (305, 290, 290)
_num_chunks: 1872 _X.shape: (1872, 64, 64, 64) _Y.shape: (1872, 7)
processed patient: 28352e12fe29361dfd9613ed2e729192 _original shape: (322, 382, 382)
_num_chunks: 3328 _X.shape: (3328, 64, 64, 64) _Y.shape: (3328, 7)
processed patient: 7852cb521d7029ca08133476054e7bec _original shape: (335, 340, 340)
_num_chunks: 2744 _X.shape: (2744, 64, 64, 64) _Y.shape: (2744, 7)
processed patient: d5c43054ba0f66d5017a8ddfde8c8c34 _original shape: (318, 360, 360)
_num_chunks: 2925 _X.shape: (2925, 64, 64, 64) _Y.shape: (2925, 7)
processed patient: 174c5f7c33ca31443208ef873b9477e5 _original shape: (346, 340, 340)
_num_chunks: 2744 _X.shape: (2744, 64, 64, 64) _Y.shape: (2744, 7)
processed patient: d43c9dd1be361b9302c9343af09cc23e _original shape: (342, 372, 372)
_nu

In [121]:
labels = pd.read_csv(LABELS)
dsb_y = []
for i in patient_ids:
    print(i)
    try:
        label_val = int(labels.loc[labels['id'] == i, 'cancer'])
    except TypeError:
        label_val = -1
    dsb_y.append(label_val)

print(dsb_y)

0a0c32c9e08cc2ea76a71649de56be6d
7577cbd6961b0cab27f88727dcd2d6d3
28352e12fe29361dfd9613ed2e729192
7852cb521d7029ca08133476054e7bec
d5c43054ba0f66d5017a8ddfde8c8c34
174c5f7c33ca31443208ef873b9477e5
d43c9dd1be361b9302c9343af09cc23e
bc43e8a2cb05a45e73dea8c7e02f2cc1
e188bdeea72bb41d980dc2556dc8aafa
49433c1588cc078b825a0eff1dc2e816
1acbe17dc8f9f59d2fd167b2aa6c650f
a19a122fe9a790576b57c6bd5cf9ff5c
6fd582d25eeb2250c2b0996c4216deb9
b1091b5af8b47f24c4b94a9d27c813d1
bdb59c956d48bc1a263acf997926c4d3
6f229187fe608c9eacc567eb74c1458c
4e9c2cbf385db44b6f5e6618f47318d9
c67e799bcc1e2635eb9164f6e8cf75f3
65073aadb60e398d8db1806f5ea2a082
2a549c66f24af964de6f25165c55e331
[0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, 1, 0, 0, 0, -1, 0, 1, 1, 0]


In [126]:
PATIENT_ANALYSIS_NUMBER = random.randint(0,10)
PATIENT_ANALYSIS_NUMBER

7

In [127]:
for i in range(len(dsb_chunks_x)):
    np.random.seed(12345678)
    x1 = dsb_chunks_x[i].flatten()
    x2 = luna_chunks_x[PATIENT_ANALYSIS_NUMBER].flatten()
    test_results = stats.ttest_ind(x1,x2, equal_var = False)
    print("result-"+str(i), test_results)


result-0 Ttest_indResult(statistic=-9982.8205356536546, pvalue=0.0)
result-1 Ttest_indResult(statistic=-10846.142108032525, pvalue=0.0)
result-2 Ttest_indResult(statistic=-9085.8526929399395, pvalue=0.0)
result-3 Ttest_indResult(statistic=-10770.242350623386, pvalue=0.0)
result-4 Ttest_indResult(statistic=-8832.5604580663839, pvalue=0.0)
result-5 Ttest_indResult(statistic=-9214.2887661027216, pvalue=0.0)
result-6 Ttest_indResult(statistic=-8065.7442171670054, pvalue=0.0)
result-7 Ttest_indResult(statistic=-9001.6812384088607, pvalue=0.0)
result-8 Ttest_indResult(statistic=-8228.8211614381962, pvalue=0.0)
result-9 Ttest_indResult(statistic=-10830.927692217674, pvalue=0.0)
result-10 Ttest_indResult(statistic=-7662.7258070853895, pvalue=0.0)
result-11 Ttest_indResult(statistic=-9591.0554638094272, pvalue=0.0)
result-12 Ttest_indResult(statistic=-11375.972037187355, pvalue=0.0)
result-13 Ttest_indResult(statistic=-9978.1466826695341, pvalue=0.0)
result-14 Ttest_indResult(statistic=-8071.42